# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## Chain Class 基类

类继承关系：

```
Chain --> <name>Chain  # Examples: LLMChain, MapReduceChain, RouterChain
```

**代码实现：https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/base.py**

```python
# 定义一个名为Chain的基础类
class Chain(Serializable, Runnable[Dict[str, Any], Dict[str, Any]], ABC):
    """为创建结构化的组件调用序列的抽象基类。
    
    链应该用来编码对组件的一系列调用，如模型、文档检索器、其他链等，并为此序列提供一个简单的接口。
    
    Chain接口使创建应用程序变得容易，这些应用程序是：
    - 有状态的：给任何Chain添加Memory可以使它具有状态，
    - 可观察的：向Chain传递Callbacks来执行额外的功能，如记录，这在主要的组件调用序列之外，
    - 可组合的：Chain API足够灵活，可以轻松地将Chains与其他组件结合起来，包括其他Chains。
    
    链公开的主要方法是：
    - `__call__`：链是可以调用的。`__call__`方法是执行Chain的主要方式。它将输入作为一个字典接收，并返回一个字典输出。
    - `run`：一个方便的方法，它以args/kwargs的形式接收输入，并将输出作为字符串或对象返回。这种方法只能用于一部分链，不能像`__call__`那样返回丰富的输出。
    """

    # 调用链
    def invoke(
        self, input: Dict[str, Any], config: Optional[runnableConfig] = None
    ) -> Dict[str, Any]:
        """传统调用方法。"""
        return self(input, **(config or {}))

    # 链的记忆，保存状态和变量
    memory: Optional[BaseMemory] = None
    """可选的内存对象，默认为None。
    内存是一个在每个链的开始和结束时被调用的类。在开始时，内存加载变量并在链中传递它们。在结束时，它保存任何返回的变量。
    有许多不同类型的内存，请查看内存文档以获取完整的目录。"""

    # 回调，可能用于链的某些操作或事件。
    callbacks: Callbacks = Field(default=None, exclude=True)
    """可选的回调处理程序列表（或回调管理器）。默认为None。
    在对链的调用的生命周期中，从on_chain_start开始，到on_chain_end或on_chain_error结束，都会调用回调处理程序。
    每个自定义链可以选择调用额外的回调方法，详细信息请参见Callback文档。"""

    # 是否详细输出模式
    verbose: bool = Field(default_factory=_get_verbosity)
    """是否以详细模式运行。在详细模式下，一些中间日志将打印到控制台。默认值为`langchain.verbose`。"""

    # 与链关联的标签
    tags: Optional[List[str]] = None
    """与链关联的可选标签列表，默认为None。
    这些标签将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""

    # 与链关联的元数据
    metadata: Optional[Dict[str, Any]] = None
    """与链关联的可选元数据，默认为None。
    这些元数据将与对这个链的每次调用关联起来，并作为参数传递给在`callbacks`中定义的处理程序。
    你可以使用这些来例如识别链的特定实例与其用例。"""
```

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

In [2]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

llm = OpenAI(temperature=0.9, max_tokens=500, 
  base_url="https://api.132006.xyz/v1/", 
  api_key="sk-xJ9ZXHmVpGyiqGWI14EbC4E12dC24e46B4D582Be6e95445d")

In [3]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="给制造{product}的有限公司取10个好名字，并给出完整的公司名称",
)

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "性能卓越的GPU"
    }))

。
1. GPU引擎公司（GPU Engine Company）
2. 高性能GPU有限公司（High Performance GPU Limited）
3. GPU处理器公司（GPU Processing Company）
4. 图形处理技术公司（Graphic Processing Technologies Company）
5. GPU运算设计公司（GPU Computational Design Company）
6. 超高效能GPU公司（Ultra High-Performance GPU Company）
7. 3D显示GPU有限公司（3D Display GPU Limited）
8. 图形计算服务公司（Graphic Computing Services Company）
9. GPU科技有限公司（GPU Technology Limited）
10. 多媒体GPU技术有限公司（Multimedia GPU Technology Limited）


In [5]:
chain.verbose = True

In [6]:
chain.verbose

True

In [7]:
print(chain.run({
    'product': "性能卓越的GPU"
    }))



> Entering new LLMChain chain...
Prompt after formatting:
给制造性能卓越的GPU的有限公司取10个好名字，并给出完整的公司名称

> Finished chain.


1. 优众芯GPU有限公司; 
2. 晶唯芯讯GPU有限公司; 
3. 思翔芯科GPU有限公司; 
4. 高矽芯技GPU有限公司; 
5. 嘉科芯能GPU有限公司; 
6. 博冠芯星GPU有限公司; 
7. 博意芯科GPU有限公司; 
8. 美核芯技GPU有限公司; 
9. 合邦芯动GPU有限公司; 
10. 台晶芯科GPU有限公司。


## Sequential Chain

串联式调用语言模型（将一个调用的输出作为另一个调用的输入）。

顺序链（Sequential Chain ）允许用户连接多个链并将它们组合成执行特定场景的流水线（Pipeline）。有两种类型的顺序链：

- SimpleSequentialChain：最简单形式的顺序链，每个步骤都具有单一输入/输出，并且一个步骤的输出是下一个步骤的输入。
- SequentialChain：更通用形式的顺序链，允许多个输入/输出。

### 使用 SimpleSequentialChain 实现戏剧摘要和评论（单输入/单输出）

![](../images/simple_sequential_chain_0.png)

In [8]:
# 这是一个 LLMChain，用于根据剧目的标题撰写简介。

llm = OpenAI(temperature=0.7, max_tokens=1000, 
  base_url="https://api.132006.xyz/v1/", 
  api_key="sk-xJ9ZXHmVpGyiqGWI14EbC4E12dC24e46B4D582Be6e95445d")

template = """你是一位剧作家。根据戏剧的标题，你的任务是为该标题写一个简介。

标题：{title}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title"], template=template)
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template)

In [9]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。
# llm = OpenAI(temperature=0.7, max_tokens=1000)
template = """你是《纽约时报》的戏剧评论家。根据剧情简介，你的工作是为该剧撰写一篇评论。

剧情简介：
{synopsis}

以下是来自《纽约时报》戏剧评论家对上述剧目的评论："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template)

![](../images/simple_sequential_chain_1.png)

In [10]:
# 这是一个SimpleSequentialChain，按顺序运行这两个链
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(chains=[synopsis_chain, review_chain], verbose=True)

In [11]:
review = overall_chain.run("三体人不是无法战胜的")



> Entering new SimpleSequentialChain chain...

《三体人不是无法战胜的》是一部关于一群有着不同背景的年轻人如何挑战和战胜一个可怕的三体人的故事。他们发现，尽管三体人拥有强大的力量，但他们也有自己的弱点，因此他们决定攻克它们。在经历了艰辛的冒险，他们最终成功了，同时发现了自己的力量和勇气。


《三体人不是无法战胜的》是一部令人振奋的戏剧，它向观众展示了一群有着不同背景的年轻人如何勇敢地挑战和战胜一个可怕的三体人的故事。这部戏剧不仅展示了他们的勇气，更重要的是，它也展示了他们如何一起协作，发掘自己的力量，克服困难，并最终取得胜利。它不仅为观众提供了一个惊险的冒险，更激励着他们去克服困难，勇敢地挑战自己的极限。它是一部令人感动的戏剧，值得所有人去观看。

> Finished chain.


In [12]:
review = overall_chain.run("星球大战第九季")



> Entering new SimpleSequentialChain chain...


星球大战第九季围绕着新一代的强大英雄展开，他们将面临一系列未知的威胁，而这些威胁可能来自可怕的新敌人或未知的宇宙力量。他们将在宇宙中完成一系列惊险的冒险，完成终极的任务，拯救宇宙免遭毁灭，并发现他们自己的真正使命。他们还将得到一系列神秘的武器，以及具有神秘力量的古老遗物，可以帮助他们将宇宙保存下来。


星球大战第九季是一部令人兴奋的冒险戏剧，充满了令人惊叹的奇幻场景，精彩的战斗场面和吸引人的英雄故事，将观众带入一个虚构的宇宙。剧中的新一代英雄将面临一系列新的威胁，而他们的任务是保护宇宙免遭毁灭。它还拥有一系列令人兴奋的悬念，以及一系列神秘的武器和古老遗物，让观众进入一个神奇而迷人的宇宙。这部剧将让观众参与一场令人兴奋的冒险，观众将一起探索一个未知的宇宙，探寻真正的使命。总之，星球大战第九季是一部出色的冒险戏剧，值得观看。

> Finished chain.


### 使用 SequentialChain 实现戏剧摘要和评论（多输入/多输出）

![](../images/sequential_chain_0.png)

In [31]:
# # 这是一个 LLMChain，根据剧名和设定的时代来撰写剧情简介。
llm = OpenAI(temperature=.7, max_tokens=1000, model_name="gpt-3.5-turbo-instruct",
  base_url="https://api.132006.xyz/v1/", 
  api_key="sk-xJ9ZXHmVpGyiqGWI14EbC4E12dC24e46B4D582Be6e95445d")
template = """你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：{title}
时代：{era}
剧作家：以下是对上述戏剧的简介："""

prompt_template = PromptTemplate(input_variables=["title", "era"], template=template)
# output_key
synopsis_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="synopsis", verbose=True)

In [32]:
# 这是一个LLMChain，用于根据剧情简介撰写一篇戏剧评论。

template = """你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：
{synopsis}

来自《纽约时报》戏剧评论家对上述剧目的评价："""

prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review", verbose=True)

In [33]:
from langchain.chains import SequentialChain

m_overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True)

In [34]:
m_overall_chain({"title":"雄纠纠气昂昂跨过鸭绿江", "era": "二十世纪的中国"})



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
你是一位剧作家。根据戏剧的标题和设定的时代，你的任务是为该标题写一个简介。

标题：雄纠纠气昂昂跨过鸭绿江
时代：二十世纪的中国
剧作家：以下是对上述戏剧的简介：

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
你是《纽约时报》的戏剧评论家。根据该剧的剧情简介，你需要撰写一篇关于该剧的评论。

剧情简介：


《雄纠纠气昂昂跨过鸭绿江》是一部讲述二十世纪中国历史中关键时刻的戏剧作品。故事发生在1949年，中华人民共和国成立前夕，中国共产党领导的人民解放军正面临着最艰巨的挑战：跨越鸭绿江，进军朝鲜半岛，与美国支持的联合国军作战。这场军事行动不仅决定着中国的命运，也影响着整个亚洲的未来。剧中，观众将会见证军队的壮举，感受到每一位战士的英勇牺牲和不屈不挠的精神。同时，也能够看到政治斗争的复杂性和决策者们的艰难抉择。这部戏剧将会带领观众穿越时空，感受那个时代最激动人心的瞬间，让我们一起见证中国人民的奇迹！

来自《纽约时报》戏剧评论家对上述剧目的评价：

> Finished chain.

> Finished chain.


{'title': '雄纠纠气昂昂跨过鸭绿江',
 'era': '二十世纪的中国',
 'synopsis': '\n\n《雄纠纠气昂昂跨过鸭绿江》是一部讲述二十世纪中国历史中关键时刻的戏剧作品。故事发生在1949年，中华人民共和国成立前夕，中国共产党领导的人民解放军正面临着最艰巨的挑战：跨越鸭绿江，进军朝鲜半岛，与美国支持的联合国军作战。这场军事行动不仅决定着中国的命运，也影响着整个亚洲的未来。剧中，观众将会见证军队的壮举，感受到每一位战士的英勇牺牲和不屈不挠的精神。同时，也能够看到政治斗争的复杂性和决策者们的艰难抉择。这部戏剧将会带领观众穿越时空，感受那个时代最激动人心的瞬间，让我们一起见证中国人民的奇迹！',
 'review': '\n\n《雄纠纠气昂昂跨过鸭绿江》是一部令人激动的戏剧作品，它将观众带入了二十世纪中国历史的关键时刻。通过讲述1949年中国共产党领导的人民解放军跨越鸭绿江的故事，这部剧揭示了一段重要的历史事件，同时也展现了壮举背后的人性故事。\n\n剧中，观众能够感受到每一位战士的英勇牺牲和不屈不挠的精神。他们为了国家和民族的命运，勇敢地跨越鸭绿江，进军朝鲜半岛。这场军事行动不仅决定着中国的命运，也影响着整个亚洲的未来。剧中的场景和动作设计也十分精彩，让观众仿佛身临其境，感受到了当时的紧张氛围。\n\n除了战争场面，剧中也展现了政治斗争的复杂性和决策者们的艰难抉择。这让观众更加深入地了解了当时的历史背景和政治环境。演员们的精湛表演也让这些复杂的政治角色栩栩如生，让观众在剧场内得以一窥当时的政治风云。\n\n最让人感动的是，这部戏剧还带领观众穿越时空，感受那个时代最激动人心的瞬间。从中国共产党领导的人民解放军的壮举到政治斗争的复杂性，再到每个战士的个人故事，这部剧将观众带入了一个充满感情和历史的世界。\n\n总的来说，《雄纠纠气昂昂跨过鸭绿江》是一部令人感动的戏剧作品。它让观众在欣赏精彩的剧情同时，也可以更加深入地了解中国近现代史的关键时刻。这部剧值得一看，让我们一起见证中国人民的奇迹！'}

### Homework

#### 使用 OutputParser 优化 overall_chain 输出格式，区分 synopsis_chain 和 review_chain 的结果